In [1]:
import math
import pickle

import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm

tqdm.pandas()

/Users/far/.pyenv/versions/miniconda3-4.3.30/envs/spec-top-econ-data-vis/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  
/Users/far/.pyenv/versions/miniconda3-4.3.30/envs/spec-top-econ-data-vis/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Utils

In [2]:
def calc_distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [3]:
def load_pickle(path):
    with open(path, 'rb') as pickle_file:
        return pickle.load(pickle_file)

## Crime

### Filter out some data

In [ ]:
df = pd.read_csv('Crimes_-_2001_to_present.csv')
df.head()

In [ ]:
display(df.info())
display(df.describe())

In [ ]:
df_2 = df[df['Year'] >= 2010].reset_index().drop(columns=['index'])
display(df_2.head())

In [ ]:
df_2.to_csv('Crimes_-_2010_to_present.csv', index=False)
del df

### Police Station

In [4]:
# load filtered crime data
df = pd.read_csv('Crimes_-_2010_to_present.csv')

In [5]:
df['District'] = df['District'].fillna(0).astype(int).astype(str)
display(df.head())
display(df.info())

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,...,21.0,73.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
1,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,False,False,...,18.0,70.0,05,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
2,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,False,False,...,20.0,42.0,06,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
3,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,...,42.0,32.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
4,11227517,JB138481,02/10/2013 12:00:00 AM,071XX S LAFAYETTE AVE,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,False,...,6.0,69.0,02,NaN,NaN,2013,02/11/2018 03:57:41 PM,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3041632 entries, 0 to 3041631
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Case Number           object 
 2   Date                  object 
 3   Block                 object 
 4   IUCR                  object 
 5   Primary Type          object 
 6   Description           object 
 7   Location Description  object 
 8   Arrest                bool   
 9   Domestic              bool   
 10  Beat                  int64  
 11  District              object 
 12  Ward                  float64
 13  Community Area        float64
 14  FBI Code              object 
 15  X Coordinate          float64
 16  Y Coordinate          float64
 17  Year                  int64  
 18  Updated On            object 
 19  Latitude              float64
 20  Longitude             float64
 21  Location              object 
dtypes: bool(2), float64(6), int64(3), object(1

None

In [6]:
# load police station data
df_police_station = pd.read_csv('Police_Stations.csv')
display(df_police_station.head())
display(df_police_station.info())

,DISTRICT,DISTRICT NAME,ADDRESS,CITY,STATE,ZIP,WEBSITE,PHONE,FAX,TTY,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
0,1,Central,1718 S State St,Chicago,IL,60616,http://home.chicagopolice.org/community/distri...,312-745-4290,312-745-3694,312-745-3693,1176569.052,1891771.704,41.858373,-87.627356,"(41.8583725929, -87.627356171)"
1,6,Gresham,7808 S Halsted St,Chicago,IL,60620,http://home.chicagopolice.org/community/distri...,312-745-3617,312-745-3649,312-745-3639,1172283.013,1853022.646,41.752137,-87.644229,"(41.7521368378, -87.6442289066)"
2,11,Harrison,3151 W Harrison St,Chicago,IL,60612,http://home.chicagopolice.org/community/distri...,312-746-8386,312-746-4281,312-746-5151,1155244.069,1897148.755,41.873582,-87.705488,"(41.8735822883, -87.705488126)"
3,16,Jefferson Park,5151 N Milwaukee Ave,Chicago,IL,60630,http://home.chicagopolice.org/community/distri...,312-742-4480,312-742-4421,312-742-4423,1138480.758,1933660.473,41.974094,-87.766149,"(41.9740944511, -87.7661488432)"
4,Headquarters,Headquarters,3510 S Michigan Ave,Chicago,IL,60653,http://home.chicagopolice.org,NaN,NaN,NaN,1177731.401,1881697.404,41.830702,-87.623395,"(41.8307016873, -87.6233953459)"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   DISTRICT       23 non-null     object 
 1   DISTRICT NAME  23 non-null     object 
 2   ADDRESS        23 non-null     object 
 3   CITY           23 non-null     object 
 4   STATE          23 non-null     object 
 5   ZIP            23 non-null     int64  
 6   WEBSITE        23 non-null     object 
 7   PHONE          22 non-null     object 
 8   FAX            22 non-null     object 
 9   TTY            22 non-null     object 
 10  X COORDINATE   23 non-null     float64
 11  Y COORDINATE   23 non-null     float64
 12  LATITUDE       23 non-null     float64
 13  LONGITUDE      23 non-null     float64
 14  LOCATION       23 non-null     object 
dtypes: float64(4), int64(1), object(10)
memory usage: 2.8+ KB


None

In [7]:
# rename police station columns
df_police_station = (
    df_police_station
    .rename(
        columns={
            'ADDRESS': 'Station_Address',
            'CITY': 'Station_City',
                    'CITY': 'Station_City',
                    'STATE': 'Station_State',
                    'ZIP': 'Station_Zip',
                    'LATITUDE': 'Station_Latitude',
                    'LONGITUDE': 'Station_Longtitude',
                    'LOCATION': 'Station_Location',
        }
    )
    .drop(columns=['WEBSITE', 'PHONE', 'FAX', 'TTY', 'X COORDINATE', 'Y COORDINATE']))
display(df_police_station.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   DISTRICT            23 non-null     object 
 1   DISTRICT NAME       23 non-null     object 
 2   Station_Address     23 non-null     object 
 3   Station_City        23 non-null     object 
 4   Station_State       23 non-null     object 
 5   Station_Zip         23 non-null     int64  
 6   Station_Latitude    23 non-null     float64
 7   Station_Longtitude  23 non-null     float64
 8   Station_Location    23 non-null     object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1.7+ KB


None

In [8]:
# join police station
df = pd.merge(df, df_police_station, how='left', left_on='District', right_on='DISTRICT')

In [9]:
display(df.head())
display(df.info())

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Location,DISTRICT,DISTRICT NAME,Station_Address,Station_City,Station_State,Station_Zip,Station_Latitude,Station_Longtitude,Station_Location
0,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,...,NaN,22,Morgan Park,1900 W Monterey Ave,Chicago,IL,60643.0,41.691435,-87.668520,"(41.6914347795, -87.6685203937)"
1,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,False,False,...,NaN,8,Chicago Lawn,3420 W 63rd St,Chicago,IL,60629.0,41.778987,-87.708864,"(41.778987189, -87.7088638153)"
2,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,False,False,...,NaN,3,Grand Crossing,7040 S Cottage Grove Ave,Chicago,IL,60637.0,41.766431,-87.605748,"(41.7664308925, -87.6057478606)"
3,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,...,NaN,1,Central,1718 S State St,Chicago,IL,60616.0,41.858373,-87.627356,"(41.8583725929, -87.627356171)"
4,11227517,JB138481,02/10/2013 12:00:00 AM,071XX S LAFAYETTE AVE,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,False,...,NaN,7,Englewood,1438 W 63rd St,Chicago,IL,60636.0,41.779632,-87.660887,"(41.7796315359, -87.6608870173)"


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3041632 entries, 0 to 3041631
Data columns (total 31 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Case Number           object 
 2   Date                  object 
 3   Block                 object 
 4   IUCR                  object 
 5   Primary Type          object 
 6   Description           object 
 7   Location Description  object 
 8   Arrest                bool   
 9   Domestic              bool   
 10  Beat                  int64  
 11  District              object 
 12  Ward                  float64
 13  Community Area        float64
 14  FBI Code              object 
 15  X Coordinate          float64
 16  Y Coordinate          float64
 17  Year                  int64  
 18  Updated On            object 
 19  Latitude              float64
 20  Longitude             float64
 21  Location              object 
 22  DISTRICT              object 
 23  DISTRIC

None

In [ ]:
# skip on second run
distances = []

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    lat, lng, lat2, lng2 = row['Latitude'], row['Longitude'], row['LATITUDE'], row['LONGITUDE']
    if np.isnan(lat) or np.isnan(lng) or np.isnan(lat2) or np.isnan(lng2):
        distances.append(np.nan)
        continue
    distance = calc_distance((lat, lng), (lat2, lng2))
    distances.append(distance)
    
with open('pickle/case_police_distances.pickle', 'wb') as pickle_file:
    pickle_file.write(pickle.dumps(distances))

In [12]:
with open('pickle/case_police_distances.pickle', 'rb') as pickle_file:
    distances = pickle.load(pickle_file)
    
df['Distance_from_police_station'] = distances
display(df.head())

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,DISTRICT,DISTRICT NAME,Station_Address,Station_City,Station_State,Station_Zip,Station_Latitude,Station_Longtitude,Station_Location,Distance_from_police_station
0,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,...,22,Morgan Park,1900 W Monterey Ave,Chicago,IL,60643.0,41.691435,-87.668520,"(41.6914347795, -87.6685203937)",NaN
1,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,False,False,...,8,Chicago Lawn,3420 W 63rd St,Chicago,IL,60629.0,41.778987,-87.708864,"(41.778987189, -87.7088638153)",NaN
2,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,False,False,...,3,Grand Crossing,7040 S Cottage Grove Ave,Chicago,IL,60637.0,41.766431,-87.605748,"(41.7664308925, -87.6057478606)",NaN
3,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,...,1,Central,1718 S State St,Chicago,IL,60616.0,41.858373,-87.627356,"(41.8583725929, -87.627356171)",NaN
4,11227517,JB138481,02/10/2013 12:00:00 AM,071XX S LAFAYETTE AVE,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,False,...,7,Englewood,1438 W 63rd St,Chicago,IL,60636.0,41.779632,-87.660887,"(41.7796315359, -87.6608870173)",NaN


In [ ]:
# skip on second run

import datetime

day_names = []

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    date = datetime.datetime.strptime(row['Date'], '%m/%d/%Y %I:%M:%S %p')
    day_name = date.strftime("%a")
    day_names.append(day_name)

with open('pickle/day_names.pickle', 'wb') as pickle_file:
    pickle_file.write(pickle.dumps(day_names))

In [13]:
with open('pickle/day_names.pickle', 'rb') as pickle_file:
    day_names = pickle.load(pickle_file)
    
df['Day_name'] = day_names
display(df.head())

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,DISTRICT NAME,Station_Address,Station_City,Station_State,Station_Zip,Station_Latitude,Station_Longtitude,Station_Location,Distance_from_police_station,Day_name
0,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,...,Morgan Park,1900 W Monterey Ave,Chicago,IL,60643.0,41.691435,-87.668520,"(41.6914347795, -87.6685203937)",NaN,Sun
1,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,False,False,...,Chicago Lawn,3420 W 63rd St,Chicago,IL,60629.0,41.778987,-87.708864,"(41.778987189, -87.7088638153)",NaN,Tue
2,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,False,False,...,Grand Crossing,7040 S Cottage Grove Ave,Chicago,IL,60637.0,41.766431,-87.605748,"(41.7664308925, -87.6057478606)",NaN,Sat
3,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,...,Central,1718 S State St,Chicago,IL,60616.0,41.858373,-87.627356,"(41.8583725929, -87.627356171)",NaN,Sat
4,11227517,JB138481,02/10/2013 12:00:00 AM,071XX S LAFAYETTE AVE,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,False,...,Englewood,1438 W 63rd St,Chicago,IL,60636.0,41.779632,-87.660887,"(41.7796315359, -87.6608870173)",NaN,Sun


In [14]:
df.to_csv('csv/Crimes_with_station.csv', index=False)

### Socio Community

Join socioeconomic indicators with crime

https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2?fbclid=IwAR0SrKYTiVC03m0M8XrmXeGIE7RneEy2LzMU4XWqbzFANuvTogJsgF6wusA

In [15]:
df_socio = pd.read_csv('csv/Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv')
display(df_socio.head())
display(df_socio.info())

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 9 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Community Area Number                         77 non-null     float64
 1   COMMUNITY AREA NAME                           78 non-null     object 
 2   PERCENT OF HOUSING CROWDED                    78 non-null     float64
 3   PERCENT HOUSEHOLDS BELOW POVERTY              78 non-null     float64
 4   PERCENT AGED 16+ UNEMPLOYED                   78 non-null     float64
 5   PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA  78 non-null     float64
 6   PERCENT AGED UNDER 18 OR OVER 64              78 non-null     float64
 7   PER CAPITA INCOME                             78 non-null     int64  
 8   HARDSHIP INDEX                                77 non-null     float64
dtypes: float64(7), int64(1), object(1)
memory usage: 5.6+ KB


None

In [16]:
# join police station
df_socio_merged = pd.merge(df, df_socio, how='left', left_on='Community Area', right_on='Community Area Number')
display(df_socio_merged.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3041632 entries, 0 to 3041631
Data columns (total 42 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   ID                                            int64  
 1   Case Number                                   object 
 2   Date                                          object 
 3   Block                                         object 
 4   IUCR                                          object 
 5   Primary Type                                  object 
 6   Description                                   object 
 7   Location Description                          object 
 8   Arrest                                        bool   
 9   Domestic                                      bool   
 10  Beat                                          int64  
 11  District                                      object 
 12  Ward                                          float64
 1

None

In [18]:
del df_socio

### Grocery Store

In [24]:
# load grocery store data
df_store = pd.read_csv('csv/Grocery_Stores_-_2013.csv')
display(df_store.head())
display(df_store.info())

,STORE NAME,LICENSE ID,ACCOUNT NUMBER,SQUARE FEET,BUFFER SIZE,ADDRESS,ZIP CODE,COMMUNITY AREA NAME,COMMUNITY AREA,WARD,CENSUS TRACT,CENSUS BLOCK,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
0,BOB'S LIQUORS & GROCERIES,957.0,86492.0,10001.0,A,5069 W MADISON ST,60644,AUSTIN,25,28,17031252201,1.703125e+14,1.142425e+06,1.899505e+06,41.880296,-87.752494,"(41.8802960895, -87.7524941982)"
1,QUICK STOP FOOD MART,39791.0,3604.0,3000.0,B,2751 W PRATT BLVD,60645,WEST RIDGE,2,50,17031020601,1.703102e+14,1.156543e+06,1.944959e+06,42.004750,-87.699423,"(42.0047500731, -87.6994227766)"
2,DOMINICK'S #147/1147,41674.0,5033.0,68000.0,A,1340 S CANAL ST,60607,NEAR WEST SIDE,28,2,17031841900,1.703184e+14,1.173296e+06,1.893986e+06,41.864521,-87.639304,"(41.8645211937, -87.6393040996)"
3,CERMAK PRODUCE,1515206.0,279468.0,25000.0,A,5220 S PULASKI RD,60632,WEST ELSDON,62,23,17031620200,1.703162e+14,1.150549e+06,1.869569e+06,41.797992,-87.723444,"(41.79799187, -87.7234437408)"
4,WHOLE FOODS MARKET,54059.0,86298.0,4501.0,B,3300 N ASHLAND AVE,60657,LAKE VIEW,6,44,17031062500,1.703106e+14,1.165028e+06,1.921997e+06,41.941566,-87.668860,"(41.9415658578, -87.6688604029)"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   STORE NAME           506 non-null    object 
 1   LICENSE ID           505 non-null    float64
 2   ACCOUNT NUMBER       505 non-null    float64
 3   SQUARE FEET          491 non-null    float64
 4   BUFFER SIZE          506 non-null    object 
 5   ADDRESS              506 non-null    object 
 6   ZIP CODE             506 non-null    int64  
 7   COMMUNITY AREA NAME  506 non-null    object 
 8   COMMUNITY AREA       506 non-null    int64  
 9   WARD                 506 non-null    int64  
 10  CENSUS TRACT         506 non-null    int64  
 11  CENSUS BLOCK         505 non-null    float64
 12  X COORDINATE         506 non-null    float64
 13  Y COORDINATE         506 non-null    float64
 14  LATITUDE             506 non-null    float64
 15  LONGITUDE            506 non-null    flo

None

In [25]:
df_store_count = df_store.groupby(by='COMMUNITY AREA')[['STORE NAME']].count()
df_store_count.rename(columns={'STORE NAME': 'Grocery Store Count'}, inplace=True)
display(df_store_count)

,Grocery Store Count
COMMUNITY AREA,
1,13
2,18
3,15
4,4
5,6
...,...
73,2
74,3
75,7


In [26]:
# join grocery store
df_grocery_merged = pd.merge(df_socio_merged, df_store_count, how='left', left_on='Community Area', right_index=True) 
display(df_grocery_merged)
display(df_grocery_merged.info())

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,Grocery Store Count
0,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,...,73.0,Washington Height,1.1,16.9,20.8,13.7,42.6,19713.0,48.0,2.0
1,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,False,False,...,70.0,Ashburn,4.0,10.4,11.7,17.7,36.9,23482.0,37.0,3.0
2,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,False,False,...,42.0,Woodlawn,2.9,30.7,23.4,16.5,36.1,18672.0,58.0,5.0
3,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,...,32.0,Loop,1.5,14.7,5.7,3.1,13.5,65526.0,3.0,4.0
4,11227517,JB138481,02/10/2013 12:00:00 AM,071XX S LAFAYETTE AVE,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,False,...,69.0,Greater Grand Crossing,3.6,29.6,23.0,16.5,41.0,17285.0,66.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041627,11700926,JC279725,05/26/2019 05:13:00 PM,036XX W DOUGLAS BLVD,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,APARTMENT,False,True,...,29.0,North Lawndale,7.4,43.1,21.2,27.6,42.7,12034.0,87.0,4.0
3041628,24560,JC279072,05/26/2019 06:48:00 AM,013XX W HASTINGS ST,0110,HOMICIDE,FIRST DEGREE MURDER,CHA PARKING LOT,True,False,...,28.0,Near West Side,3.8,20.6,10.7,9.6,22.2,44689.0,15.0,9.0
3041629,24559,JC278908,05/26/2019 02:11:00 AM,013XX W HASTINGS ST,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,False,False,...,28.0,Near West Side,3.8,20.6,10.7,9.6,22.2,44689.0,15.0,9.0
3041630,11707734,JC287730,07/01/2014 07:30:00 AM,063XX S NORMAL BLVD,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,False,False,...,68.0,Englewood,3.8,46.6,28.0,28.5,42.5,11888.0,94.0,8.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3041632 entries, 0 to 3041631
Data columns (total 43 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   ID                                            int64  
 1   Case Number                                   object 
 2   Date                                          object 
 3   Block                                         object 
 4   IUCR                                          object 
 5   Primary Type                                  object 
 6   Description                                   object 
 7   Location Description                          object 
 8   Arrest                                        bool   
 9   Domestic                                      bool   
 10  Beat                                          int64  
 11  District                                      object 
 12  Ward                                          float64
 1

None

In [21]:
df_grocery_merged.to_csv('csv/Crimes_joined.csv', index=False)

In [27]:
del df_store
del df

## Street Light

In [30]:
# use crime data only 2017 - present
df_crime = pd.read_csv('csv/Crimes_-_2010_to_present.csv')
df_crime = df_crime[df_crime['Year'] >= 2017].reset_index()
df_crime['District'] = df_crime['District'].fillna(0).astype(int).astype(str)
display(df_crime.head())
display(df_crime.info())

,index,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,0,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,...,21.0,73.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
1,1,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,False,...,18.0,70.0,05,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
2,2,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,False,...,20.0,42.0,06,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
3,3,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,...,42.0,32.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
4,6,11227508,JB146365,01/01/2017 12:01:00 AM,027XX S WHIPPLE ST,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,False,...,12.0,30.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866825 entries, 0 to 866824
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   index                 866825 non-null  int64  
 1   ID                    866825 non-null  int64  
 2   Case Number           866825 non-null  object 
 3   Date                  866825 non-null  object 
 4   Block                 866825 non-null  object 
 5   IUCR                  866825 non-null  object 
 6   Primary Type          866825 non-null  object 
 7   Description           866825 non-null  object 
 8   Location Description  863254 non-null  object 
 9   Arrest                866825 non-null  bool   
 10  Domestic              866825 non-null  bool   
 11  Beat                  866825 non-null  int64  
 12  District              866825 non-null  object 
 13  Ward                  866804 non-null  float64
 14  Community Area        866825 non-null  float64
 15  

None

In [31]:
# load street light data
df_light = pd.read_csv('csv/311_Service_Requests_-_Street_Lights_-_One_Out_-_Historical.csv')
display(df_light.head())
display(df_light.info())

,Creation Date,Status,Completion Date,Service Request Number,Type of Service Request,Street Address,ZIP Code,X Coordinate,Y Coordinate,Ward,Police District,Community Area,Latitude,Longitude,Location
0,04/04/2013,Completed,04/04/2013,13-00376850,Street Light - 1/Out,1 W LAKE,NaN,NaN,NaN,42.0,1.0,32.0,NaN,NaN,NaN
1,02/12/2013,Completed,02/12/2013,13-00165046,Street Light - 1/Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02/15/2013,Completed,02/15/2013,13-00178644,Street Light - 1/Out,1 W LAKE,NaN,NaN,NaN,42.0,1.0,32.0,NaN,NaN,NaN
3,02/18/2013,Completed,02/18/2013,13-00186181,Street Light - 1/Out,1 W LAKE,NaN,NaN,NaN,42.0,1.0,32.0,NaN,NaN,NaN
4,02/22/2013,Completed,02/22/2013,13-00204983,Street Light - 1/Out,1 W LAK,NaN,NaN,NaN,42.0,1.0,32.0,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490937 entries, 0 to 490936
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Creation Date            490937 non-null  object 
 1   Status                   490937 non-null  object 
 2   Completion Date          486817 non-null  object 
 3   Service Request Number   490937 non-null  object 
 4   Type of Service Request  490937 non-null  object 
 5   Street Address           490936 non-null  object 
 6   ZIP Code                 489919 non-null  float64
 7   X Coordinate             490478 non-null  float64
 8   Y Coordinate             490478 non-null  float64
 9   Ward                     490515 non-null  float64
 10  Police District          490518 non-null  float64
 11  Community Area           490520 non-null  float64
 12  Latitude                 490478 non-null  float64
 13  Longitude                490478 non-null  float64
 14  Loca

None

In [32]:
# convert datetime string to datetime object
df_light['Creation Date'] = pd.to_datetime(df_light['Creation Date'])
df_light['Completion Date'] = pd.to_datetime(df_light['Completion Date'])

In [33]:
# set index to 'Police District' for lookup
df_light['Police District'] = df_light['Police District'].fillna(0).astype(int).astype(str)
df_light.set_index('Police District', inplace=True)
display(df_light.head())
display(df_light.info())

,Creation Date,Status,Completion Date,Service Request Number,Type of Service Request,Street Address,ZIP Code,X Coordinate,Y Coordinate,Ward,Community Area,Latitude,Longitude,Location
Police District,,,,,,,,,,,,,,
1,2013-04-04,Completed,2013-04-04,13-00376850,Street Light - 1/Out,1 W LAKE,NaN,NaN,NaN,42.0,32.0,NaN,NaN,NaN
0,2013-02-12,Completed,2013-02-12,13-00165046,Street Light - 1/Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-02-15,Completed,2013-02-15,13-00178644,Street Light - 1/Out,1 W LAKE,NaN,NaN,NaN,42.0,32.0,NaN,NaN,NaN
1,2013-02-18,Completed,2013-02-18,13-00186181,Street Light - 1/Out,1 W LAKE,NaN,NaN,NaN,42.0,32.0,NaN,NaN,NaN
1,2013-02-22,Completed,2013-02-22,13-00204983,Street Light - 1/Out,1 W LAK,NaN,NaN,NaN,42.0,32.0,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Index: 490937 entries, 1 to 22
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   Creation Date            490937 non-null  datetime64[ns]
 1   Status                   490937 non-null  object        
 2   Completion Date          486817 non-null  datetime64[ns]
 3   Service Request Number   490937 non-null  object        
 4   Type of Service Request  490937 non-null  object        
 5   Street Address           490936 non-null  object        
 6   ZIP Code                 489919 non-null  float64       
 7   X Coordinate             490478 non-null  float64       
 8   Y Coordinate             490478 non-null  float64       
 9   Ward                     490515 non-null  float64       
 10  Community Area           490520 non-null  float64       
 11  Latitude                 490478 non-null  float64       
 12  Longitude                

None

In [ ]:
# count street light down for each case

import datetime
import os


def calc_street_light_count():
    if os.path.isfile('pickle/light_count_dict.pickle'):
        with open('pickle/light_count_dict.pickle', 'rb') as pickle_file:
            light_count_dict = pickle.load(pickle_file)
    else:
        light_count_dict = {
            'have_lights': [],
            'count_50meters': [],
            'count_100meters': [],
            'count_200meters': [],
            'count_500meters': [],
            'count_1000meters': [],
            'nearests': []
        }

    for index, row in tqdm(df_crime_temp.iterrows(), total=df_crime_temp.shape[0]):
        if index < len(light_count_dict['count_50meters']):
            continue

        have_light = False
        count_50meter = 0
        count_100meter = 0
        count_200meter = 0
        count_500meter = 0
        count_1000meter = 0
        nearest = np.nan

        lat, lng = row['Latitude'], row['Longitude']
        if np.isnan(lat) or np.isnan(lng):
            pass
        else:
            district = row['District']
            date = datetime.datetime.strptime(
                row['Date'], '%m/%d/%Y %I:%M:%S %p')
            df_temp = df_light.loc[district]
            lights = df_temp[(df_temp['Creation Date'] < date) &
                             (df_temp['Completion Date'] > date)]

            if len(lights) > 0:

                for _, light_row in lights.iterrows():
                    lat2, lng2 = light_row['Latitude'], light_row['Longitude']
                    if np.isnan(lat2) or np.isnan(lng2):
                        continue

                    distance = calc_distance((lat, lng), (lat2, lng2))

                    if np.isnan(nearest) or nearest > distance:
                        nearest = distance

                    if distance <= 0.05:
                        count_50meter += 1

                    if distance <= 0.1:
                        count_100meter += 1
                        have_light = True

                    if distance <= 0.2:
                        count_200meter += 1
                        have_light = True

                    if distance <= 0.5:
                        count_500meter += 1

                    if distance <= 1:
                        count_1000meter += 1
    #                     print(distance)

        light_count_dict['have_lights'].append(have_light)
        light_count_dict['count_50meters'].append(count_50meter)
        light_count_dict['count_100meters'].append(count_100meter)
        light_count_dict['count_200meters'].append(count_200meter)
        light_count_dict['count_500meters'].append(count_500meter)
        light_count_dict['count_1000meters'].append(count_1000meter)
        light_count_dict['nearests'].append(nearest)

        if index % 1000 == 0:
            with open('pickle/light_count_dict.pickle', 'wb') as pickle_file:
                pickle_file.write(pickle.dumps(light_count_dict))

    with open('pickle/light_count_dict.pickle', 'wb') as pickle_file:
        pickle_file.write(pickle.dumps(light_count_dict))

# Deprecated, use street_light.py instead for faster speed
# calc_street_light_count()

In [ ]:
# with open('pickle/light_count_dict.pickle', 'rb') as pickle_file:
#     light_count_dict = pickle.load(pickle_file)
# len(light_count_dict['have_lights'])

with open('pickle/street_light_out_count.pickle', 'rb') as pickle_file:
    light_count_dict2 = pickle.load(pickle_file)
len(light_count_dict2.items())

In [ ]:
# Deprecated, skip this

# df_crime_temp['ST_Light_Out_Count_50m'] = light_count_dict['count_50meters']
# df_crime_temp['ST_Light_Out_Count_100m'] = light_count_dict['count_100meters']
# df_crime_temp['ST_Light_Out_Count_200m'] = light_count_dict['count_200meters']
# df_crime_temp['ST_Light_Out_Count_500m'] = light_count_dict['count_500meters']
# df_crime_temp['ST_Light_Out_Count_1000m'] = light_count_dict['count_1000meters']
# df_crime_temp['ST_Light_Out_Nearest_Distance'] = light_count_dict['nesrests']

# display(df_crime_temp.head())
# display(df_crime_temp.info())

In [36]:
del df_crime
del df_light

### Generate csv

In [34]:
st_dict = load_pickle('pickle/street_light_out_count.pickle')

st_df_dict = {
    'ID': [],
    'ST_Light_Out_Count_50m': [],
    'ST_Light_Out_Count_100m': [],
    'ST_Light_Out_Count_200m': [],
    'ST_Light_Out_Count_500m': [],
    'ST_Light_Out_Count_1000m': [],
    'ST_Light_Out_Nearest_Distance': []
}

for k, v in st_dict.items():
    case_id = k
    st_df_dict['ID'].append(case_id)
    st_df_dict['ST_Light_Out_Count_50m'].append(v['count_50meter'])
    st_df_dict['ST_Light_Out_Count_100m'].append(v['count_100meter'])
    st_df_dict['ST_Light_Out_Count_200m'].append(v['count_200meter'])
    st_df_dict['ST_Light_Out_Count_500m'].append(v['count_500meter'])
    st_df_dict['ST_Light_Out_Count_1000m'].append(v['count_1000meter'])
    st_df_dict['ST_Light_Out_Nearest_Distance'].append(v['nearest'])
    
st_df = pd.DataFrame(st_df_dict)
display(st_df.head())
display(st_df.info())
display(st_df.describe())

,ID,ST_Light_Out_Count_50m,ST_Light_Out_Count_100m,ST_Light_Out_Count_200m,ST_Light_Out_Count_500m,ST_Light_Out_Count_1000m,ST_Light_Out_Nearest_Distance
0,11227287,0,0,0,0,0,NaN
1,11227583,0,0,0,0,0,NaN
2,11227293,0,0,0,0,0,NaN
3,11227634,0,0,0,0,0,NaN
4,11227508,0,0,0,0,0,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866825 entries, 0 to 866824
Data columns (total 7 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ID                             866825 non-null  int64  
 1   ST_Light_Out_Count_50m         866825 non-null  int64  
 2   ST_Light_Out_Count_100m        866825 non-null  int64  
 3   ST_Light_Out_Count_200m        866825 non-null  int64  
 4   ST_Light_Out_Count_500m        866825 non-null  int64  
 5   ST_Light_Out_Count_1000m       866825 non-null  int64  
 6   ST_Light_Out_Nearest_Distance  540879 non-null  float64
dtypes: float64(1), int64(6)
memory usage: 46.3 MB


None

,ID,ST_Light_Out_Count_50m,ST_Light_Out_Count_100m,ST_Light_Out_Count_200m,ST_Light_Out_Count_500m,ST_Light_Out_Count_1000m,ST_Light_Out_Nearest_Distance
count,8.668250e+05,866825.000000,866825.000000,866825.000000,866825.000000,866825.000000,540879.000000
mean,1.140078e+07,0.051363,0.146926,0.569392,3.249989,11.050545,0.423008
std,6.431943e+05,0.352464,0.649943,1.475393,5.136449,14.408285,0.821405
min,2.305900e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000024
25%,1.112717e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.135323
50%,1.141838e+07,0.000000,0.000000,0.000000,1.000000,5.000000,0.240357
75%,1.172820e+07,0.000000,0.000000,0.000000,5.000000,18.000000,0.409748
max,1.205420e+07,31.000000,111.000000,177.000000,181.000000,217.000000,19.828267


In [35]:
st_df.to_csv('csv/Crimes_with_street_light_count.csv', index=False)

## Merge

In [37]:
df_crime = pd.read_csv('csv/Crimes_joined.csv')
display(df_crime.head())

/Users/far/.pyenv/versions/miniconda3-4.3.30/envs/spec-top-econ-data-vis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,Grocery Store Count
0,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,...,73.0,Washington Height,1.1,16.9,20.8,13.7,42.6,19713.0,48.0,2.0
1,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,False,False,...,70.0,Ashburn,4.0,10.4,11.7,17.7,36.9,23482.0,37.0,3.0
2,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,False,False,...,42.0,Woodlawn,2.9,30.7,23.4,16.5,36.1,18672.0,58.0,5.0
3,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,...,32.0,Loop,1.5,14.7,5.7,3.1,13.5,65526.0,3.0,4.0
4,11227517,JB138481,02/10/2013 12:00:00 AM,071XX S LAFAYETTE AVE,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,False,...,69.0,Greater Grand Crossing,3.6,29.6,23.0,16.5,41.0,17285.0,66.0,7.0


In [38]:
df_st = pd.read_csv('csv/Crimes_with_street_light_count.csv')
display(df_st.head())

,ID,ST_Light_Out_Count_50m,ST_Light_Out_Count_100m,ST_Light_Out_Count_200m,ST_Light_Out_Count_500m,ST_Light_Out_Count_1000m,ST_Light_Out_Nearest_Distance
0,11227287,0,0,0,0,0,NaN
1,11227583,0,0,0,0,0,NaN
2,11227293,0,0,0,0,0,NaN
3,11227634,0,0,0,0,0,NaN
4,11227508,0,0,0,0,0,NaN


In [41]:
df_merged = pd.merge(df_crime, df_st, how='left', left_on='ID', right_on='ID')

In [42]:
display(df_merged.head())
display(df_merged.info())

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,Grocery Store Count,ST_Light_Out_Count_50m,ST_Light_Out_Count_100m,ST_Light_Out_Count_200m,ST_Light_Out_Count_500m,ST_Light_Out_Count_1000m,ST_Light_Out_Nearest_Distance
0,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,...,42.6,19713.0,48.0,2.0,0.0,0.0,0.0,0.0,0.0,NaN
1,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,False,False,...,36.9,23482.0,37.0,3.0,0.0,0.0,0.0,0.0,0.0,NaN
2,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,False,False,...,36.1,18672.0,58.0,5.0,0.0,0.0,0.0,0.0,0.0,NaN
3,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,...,13.5,65526.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,NaN
4,11227517,JB138481,02/10/2013 12:00:00 AM,071XX S LAFAYETTE AVE,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,False,...,41.0,17285.0,66.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3041632 entries, 0 to 3041631
Data columns (total 49 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   ID                                            int64  
 1   Case Number                                   object 
 2   Date                                          object 
 3   Block                                         object 
 4   IUCR                                          object 
 5   Primary Type                                  object 
 6   Description                                   object 
 7   Location Description                          object 
 8   Arrest                                        bool   
 9   Domestic                                      bool   
 10  Beat                                          int64  
 11  District                                      int64  
 12  Ward                                          float64
 1

None

In [43]:
df_merged.to_csv('csv/Crimes_merged.csv', index=False)

In [47]:
!head -n 3 Crimes_merged.csv

ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,DISTRICT,DISTRICT NAME,Station_Address,Station_City,Station_State,Station_Zip,Station_Latitude,Station_Longtitude,Station_Location,Distance_from_police_station,Day_name,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME ,HARDSHIP INDEX,Grocery Store Count,ST_Light_Out_Count_50m,ST_Light_Out_Count_100m,ST_Light_Out_Count_200m,ST_Light_Out_Count_500m,ST_Light_Out_Count_1000m,ST_Light_Out_Nearest_Distance
11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,2222,22,21.0,73.0,02,,,2017,02/11/2018 03:57:41 PM,,,,22.0,Morgan Park,1900 W Monterey Ave,